In [25]:
from bs4 import BeautifulSoup
from datetime import datetime as dt
import requests
from requests import get
import pandas as pd
import numpy as np

In [26]:
url_list= ['https://www.boxofficemojo.com/chart/top_lifetime_gross/?ref_=bo_cso_ac',
           'https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=200',
           'https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=400',
           'https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=600',
           'https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=800']

links=[]
def get_movie_links(url_list):
    for url in url_list:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        for i in soup.find_all('a', class_='a-link-normal'):
            links.append(i.get('href'))
    return links

movie_links=get_movie_links(url_list)


In [27]:
movie_urls=[]
def mojo_movies(links):
    for link in links:
        if 'title' in link:
            final='https://www.boxofficemojo.com'+link
            movie_urls.append(final)
    return movie_urls
movie_number=mojo_movies(links)

In [28]:
Titles=[]
Domestic_gr=[]
Inter_gr=[]
World_gr=[]
Budget=[]
Run_time=[]
Domestic_open=[]
Rel_date=[]
Rating=[]
Genre=[]
Domestic_d=[]

def movie_stats(movie_number):
    
    for i in movie_number:
        response=requests.get(i)

        page = response.text
        soup = BeautifulSoup(page, "lxml")

        #Get title
        try:
            title=soup.find('h1', class_='a-size-extra-large').text
        except:
            title=np.nan
        Titles.append(title)

        #Get domestic gross
        try:
            domestic_gross=soup.find('span', class_='a-size-medium a-text-bold').text.split()[0]
        except:
            domestic_gross=np.nan
        Domestic_gr.append(domestic_gross)

        #International gross
        try:
            inter_gr=soup.find('span', class_='a-size-small').find_next('span', class_='a-size-small').find_next('span', class_='money').text
        except:
            inter_gr=np.nan
        Inter_gr.append(inter_gr)

        #World Gross
        try:
            world_gr = soup.find('span', class_='a-size-small').find_next('span', class_='a-size-small').find_next('span', class_='money').find_next('span', class_='a-size-small').find_next('span', class_='money').text
        except:
            world_gr = np.nan
        World_gr.append(world_gr)

        #Budget
        try:
            budget = soup.find('span', text = 'Budget').find_next('span', class_='money').text
        except:
            budget = np.nan
        Budget.append(budget)

        #Get runtime
        try:
            runtime=soup.find('span', text = 'Running Time').find_next('span').text
        except:
            runtime=np.nan
        Run_time.append(runtime)

        #Domestic Open
        try:
            domestic_open = soup.find('span', text = 'Domestic Opening').find_next('span', class_='money').text
        except:
            domestic_open=np.nan
        Domestic_open.append(domestic_open)

        #Release Date
        try:
            release_date=soup.find('span', text = 'Earliest Release Date').find_next('span').text
        except:
            release_date=np.nan
        Rel_date.append(release_date)
        
        #Get rating
        try:
            rating=soup.find('span', text = 'MPAA').find_next('span').text
        except:
            rating=np.nan
        Rating.append(rating)

        #Genre
        try:
            genre=soup.find('span', text = 'Genres').find_next('span').text
        except:
            genre=np.nan
        Genre.append(genre)



In [29]:
# Saved data to CSV
movie_stats(movie_number[:1001])
Movies=pd.read_csv('Movies.csv', index_col=0)

In [30]:
movies_dict={'Title':Titles,'Domestic_Gr':Domestic_gr,'Inter_Gr':Inter_gr,'World_Gr':World_gr, 
             'Domestic_Open':Domestic_open, 'Budget':Budget,'Rel_Date':Rel_date, 'Runtime':Run_time, 
             'Genre':Genre,'Rating':Rating}

In [31]:
Movies=pd.DataFrame(movies_dict)
# Movies['Budget'] = Movies['Budget'].str.replace(',', '')
# Movies['Budget'] = Movies['Budget'].str.replace('$', '')
# Movies.fillna(value=0, inplace=True)

# Movies['Domestic_Gr'] = Movies['Domestic_Gr'].str.replace(',', '')
# Movies['Domestic_Gr'] = Movies['Domestic_Gr'].str.replace('$', '')
# Movies.fillna(value=0, inplace=True)

# Movies['Inter_Gr'] = Movies['Inter_Gr'].str.replace(',', '')
# Movies['Inter_Gr'] = Movies['Inter_Gr'].str.replace('$', '')
# Movies.fillna(value=0, inplace=True)

# Movies['World_Gr'] = Movies['World_Gr'].str.replace(',', '')
# Movies['World_Gr'] = Movies['World_Gr'].str.replace('$', '')
# Movies.fillna(value=0, inplace=True)

# Movies['Domestic_Open'] = Movies['Domestic_Open'].str.replace(',', '')
# Movies['Domestic_Open'] = Movies['Domestic_Open'].str.replace('$', '')
# Movies.fillna(value=0, inplace=True)

# Movies['Budget'] = Movies.Budget.astype('int')
# Movies['Domestic_Gr'] = Movies.Domestic_Gr.astype('int')
# Movies['Inter_Gr'] = Movies.Inter_Gr.astype('int')
# Movies['World_Gr'] = Movies.World_Gr.astype('int')
# Movies['Domestic_Open'] = Movies.Domestic_Open.astype('int')

# Movies['Budget'] = Movies['Budget'].replace(0,Movies['Budget'].mean())
# Movies['Domestic_Gr'] = Movies['Domestic_Gr'].replace(0,Movies['Domestic_Gr'].mean())
# Movies['Inter_Gr'] = Movies['Inter_Gr'].replace(0,Movies['Inter_Gr'].mean())
# Movies['World_Gr'] = Movies['World_Gr'].replace(0,Movies['World_Gr'].mean())
# Movies['Domestic_Open'] = Movies['Domestic_Open'].replace(0,Movies['Domestic_Open'].mean())
# Movies[['World_Gr','Domestic_Open']] = Movies[['World_Gr','Domestic_Open']].astype(int)

In [32]:
# Generate dummy variables and reformat columns
dummies_genre=pd.get_dummies(Movies['Genre'])
movies = Movies.copy()
movies = pd.concat([Movies, dummies_genre], axis=1)
movies = movies.drop('Genre', axis=1)
dummies_rating=pd.get_dummies(Movies['Rating'])
movies = pd.concat([movies, dummies_rating], axis=1)
movies.columns
movies.rename(columns={'Domestic_gr':'Dom_gross','Inter_Gr':'Int_gross','World_Gr':'World_gr','Domestic_Open':'Dom_open'}, inplace=True)
movies.columns=movies.columns.str.replace(' ','')
movies.rename(columns={'Fantasy\n\nHorror\n\nMystery\n\nThriller':'Fant/Horror/Myst/Thr', 'Fantasy\n\nMystery\n\nRomance\n\nSci-Fi\n\nThriller':'Fant/Myst/Rom/Sci/Thr', 'Horror\n\nMystery':'Horror/Myst','Horror\n\nMystery\n\nThriller':'Horror/Myst/Thr','Horror\n\nSci-Fi':'Horror/Sci','Horror\n\nThriller':'Horror/Thr','Musical\n\nRomance':'Music/Rom','Mystery\n\nSci-Fi\n\nThriller':'Myst/Sci/Thr','Mystery\n\nThriller':'Myst/Thr'}, inplace=True)
movies=movies.drop('Rating', axis=1)
movies

,Title,Domestic_Gr,Int_gross,World_gr,Dom_open,Budget,Rel_Date,Runtime,Action,Action\n\nAdventure,...,Horror/Sci,Horror/Thr,Music/Rom,Myst/Sci/Thr,Myst/Thr,Approved,G,PG,PG-13,R
0,Star Wars: Episode VII - The Force Awakens (2015),"$936,662,225","$1,132,859,475","$2,069,521,700","$247,966,675","$245,000,000","December 16, 2015\n (EMEA, APAC)",2 hr 18 min,0,0,...,0,0,0,0,0,0,0,0,1,0
1,Avengers: Endgame (2019),"$858,373,000","$1,939,128,328","$2,797,501,328","$357,115,007","$356,000,000","April 24, 2019\n (21 markets)",3 hr 1 min,0,0,...,0,0,0,0,0,0,0,0,1,0
2,Avatar (2009),"$760,507,625","$2,086,738,578","$2,847,246,203","$77,025,481","$237,000,000","December 16, 2009\n (LATAM, EMEA)",2 hr 42 min,0,0,...,0,0,0,0,0,0,0,0,1,0
3,Black Panther (2018),"$700,426,566","$647,171,407","$1,347,597,973","$202,003,951",NaN,"February 13, 2018\n (EMEA, APAC)",2 hr 14 min,0,0,...,0,0,0,0,0,0,0,0,1,0
4,Avengers: Infinity War (2018),"$678,815,482","$1,369,544,272","$2,048,359,754","$257,698,183",NaN,"April 25, 2018\n (APAC, EMEA)",2 hr 29 min,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Saw III (2006),"$80,238,724","$84,635,551","$164,874,275","$33,610,391","$10,000,000","October 27, 2006\n (EMEA, Domestic)",1 hr 48 min,0,0,...,0,0,0,0,0,0,0,0,0,1
996,The Mummy (2017),"$80,227,895","$329,003,712","$409,231,607","$31,688,375","$125,000,000","June 6, 2017\n (South Korea)",1 hr 51 min,0,0,...,0,0,0,0,0,0,0,0,1,0
997,Disturbia (2007),"$80,209,692","$37,904,528","$118,114,220","$22,224,982","$20,000,000","April 12, 2007\n (APAC)",1 hr 45 min,0,0,...,0,0,0,0,0,0,0,0,1,0
998,Nacho Libre (2006),"$80,197,993","$19,057,467","$99,255,460","$28,309,599","$35,000,000","June 16, 2006\n (Domestic)",1 hr 32 min,0,0,...,0,0,0,0,0,0,0,1,0,0
